In [1]:
import os
import shutil
from datasets import Dataset, concatenate_datasets
from tqdm import tqdm

path = '/data02/users/lz/code/UICoder/datasets/c4-wash/H128-2560_C128-4096_R2/c4-format-marked/origin'
output_path = '/data02/users/lz/code/UICoder/datasets/c4-wash/H128-2560_C128-4096_R2/c4-format-marked/merged'

/data02/users/lz/miniconda3/envs/UICoder/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# len(os.listdir('/data02/users/lz/code/UICoder/datasets/c4-wash/H128-2560_C128-4096_R2/c4-format'))
# os.path.dirname('/data02/users/lz/code/UICoder/datasets/c4-wash/H128-2560_C128-4096_R2/c4-format')

194191

In [6]:
# index = 5
# start = 20000
# size = 3000

# source_path = '/data02/users/lz/code/UICoder/datasets/c4-wash/H128-2560_C128-4096_R2/c4-format'
# target_path = f'{os.path.dirname(source_path)}/c4-format-{int(size/1000)}k_{index}'
# for item in os.listdir(source_path)[start:start+size]:
#     shutil.copytree(os.path.join(source_path,item),os.path.join(target_path,item))

['1k_0', '1k_1', '1k_2', '1k_3', '6k_5', '6k_6']

In [9]:
ds_avrs = []
for batch_name in sorted(os.listdir(path)):
    print(batch_name)
    dss = []
    for one in os.listdir(os.path.join(path,batch_name)):
        ds = Dataset.load_from_disk(os.path.join(path,batch_name,one))
        dss.append(ds)

    def generator():
        for idx,item in tqdm(enumerate(dss[0]),total=len(dss[0])):
            image = item['image']
            scores = []
            item2 = {
                'image': image,
            }
            for idx2,ds in enumerate(dss):
                score = ds[idx]['score']
                item2[f'score_{idx2}'] = score
                scores.append(score)
            item2['score_avr'] = round(sum(scores)/len(scores))
            yield item2
            

    ds_avr = Dataset.from_generator(generator,)
    ds_avrs.append(ds_avr)

1k_0


Generating train split: 0 examples [00:00, ? examples/s]

100%|██████████| 1000/1000 [01:38<00:00, 10.11it/s]s/ examples]
Generating train split: 1000 examples [01:38, 10.11 examples/s]


1k_1


100%|██████████| 1000/1000 [01:46<00:00,  9.43it/s]s/ examples]
Generating train split: 1000 examples [01:46,  9.43 examples/s]


...

100%|██████████| 1000/1000 [01:43<00:00,  9.67it/s]s/ examples]
Generating train split: 1000 examples [01:43,  9.67 examples/s]


6k_5


100%|██████████| 3000/3000 [06:52<00:00,  7.27it/s]s/ examples]
Generating train split: 3000 examples [06:52,  7.27 examples/s]


6k_6


100%|██████████| 3000/3000 [06:33<00:00,  7.62it/s]s/ examples]
Generating train split: 3000 examples [06:33,  7.62 examples/s]


In [10]:
ds_merged = concatenate_datasets(ds_avrs)

In [11]:
ds_merged.save_to_disk(output_path)

Saving the dataset (3/3 shards): 100%|██████████| 10000/10000 [00:03<00:00, 2930.28 examples/s]


In [13]:
dsx = Dataset.load_from_disk(output_path)

In [19]:
len(dsx)

10000